In [20]:
import pandas as pd
import numpy as np

In [21]:
# Importação e Limpeza Inicial
df = pd.read_csv('despesasPorOrgao.csv', sep=';')
df = df.drop('Unnamed: 7', axis=1)

In [22]:
# Tratamento de Colunas Monetárias (Função para reutilização)
def limpar_valor(series):
    return (
        series.astype(str)
        .str.replace(" ", "", regex=False)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.replace("-.", "-", regex=False)
        .astype(float)
    )

In [23]:
df["Valor Empenhado"] = limpar_valor(df["Valor Empenhado"])
df["Valor Liquidado"] = limpar_valor(df["Valor Liquidado"])
df["Valor Pago"] = limpar_valor(df["Valor Pago"])
df["Valor Restos a Pagar Pagos"] = limpar_valor(df["Valor Restos a Pagar Pagos"])

In [24]:
# Tratamento de Datas e Outliers
df["Mês Ano"] = pd.to_datetime(df["Mês Ano"], format="%m/%Y")
df["Órgão/Entidade Vinculada"] = df["Órgão/Entidade Vinculada"].replace("-3", "Não informado")

In [25]:
# Tratamento de Valores Negativos (clip)
df["Valor Empenhado"] = df["Valor Empenhado"].clip(lower=0)
df["Valor Liquidado"] = df["Valor Liquidado"].clip(lower=0)
df["Valor Restos a Pagar Pagos"] = df["Valor Restos a Pagar Pagos"].clip(lower=0)

In [26]:
# Criando Métricas de Eficiência (KPIs)
# Taxa Liquidação: np.nan para evitar divisões por zero (inf)

df["Taxa Liquidação"] = np.where(df["Valor Empenhado"] == 0, np.nan, df["Valor Liquidado"] / df["Valor Empenhado"])

# Taxa de Pagamento: 0 para evitar divisões por zero
df["Taxa de Pagamento"] = np.where(df["Valor Liquidado"] > 0, df["Valor Pago"] / df["Valor Liquidado"], 0)

In [29]:
# Salvar o DataFrame Limpo para o próximo notebook
df.to_pickle('df_limpo.pkl')

# print("DataFrame limpo e com KPIs gerados.")

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3034 entries, 0 to 3033
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Mês Ano                     3034 non-null   datetime64[ns]
 1   Órgão Superior              3034 non-null   object        
 2   Órgão/Entidade Vinculada    3034 non-null   object        
 3   Valor Empenhado             3034 non-null   float64       
 4   Valor Liquidado             3034 non-null   float64       
 5   Valor Pago                  3034 non-null   float64       
 6   Valor Restos a Pagar Pagos  3034 non-null   float64       
 7   Taxa Liquidação             2748 non-null   float64       
 8   Taxa de Pagamento           3034 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 213.5+ KB
